In [1]:
import numpy as np
from numpy import linalg
import cvxopt
import cvxopt.solvers
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
from sklearn import svm
import math 

ModuleNotFoundError: No module named 'sklearn.cross_validation'

In [2]:
from sklearn import datasets
train = pd.read_csv("modifiedpima.csv", header=None)
iris = datasets.load_iris()
train=X = iris.data[:, :2]
train.head()

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [4]:
features = train.columns[0:8]
X = train[features]
y = train[8]
X.head()

,0,1,2,3,4,5,6,7
0,6.0,148.0,72.0,35.0,0.0,33.6,0.627,50.0
1,8.0,183.0,64.0,0.0,0.0,23.3,0.672,32.0
2,0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0
3,3.0,78.0,50.0,32.0,88.0,31.0,0.248,26.0
4,2.0,197.0,70.0,45.0,543.0,30.5,0.158,53.0


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)


In [8]:
print(X_train.shape,X_test.shape)

(614, 8) (154, 8)


In [9]:
X_train=np.asarray(X_train)
y_train=np.asarray(y_train)

In [10]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=100.0):
   # print(-linalg.norm(x-y)**2)
    x=np.asarray(x)
    y=np.asarray(y)
    return np.exp((-linalg.norm(x-y)**2) / (2 * (sigma ** 2)))

def gm(y_predict,y_test):
    test_min=0
    test_max=0
    pred_min=0
    pred_max=0
    y_test=np.asarray(y_test)
    for i in range(0,154):
        if(y_test[i]==1):
             test_min=test_min+1
        else:
             test_max=test_max+1
    print("y_test min",test_min)       
    print("y_test max",test_max)
    for i in range(0,154):
        if(y_predict[i]==1 and y_predict[i]==y_test[i]):
             pred_min=pred_min+1
        elif(y_predict[i]==-1 and y_predict[i]==y_test[i]):
             pred_max=pred_max+1
    print("y_pred min",pred_min)       
    print("y_pred max",pred_max)
    se=pred_min/test_min
    sp=pred_max/test_max
    print(se,sp)
    gm=math.sqrt(se*sp)
    print("GM",gm)

# FSVM using Hyperplane

In [11]:
from cvxopt import matrix
class HYP_SVM(object):

    def __init__(self, kernel=gaussian_kernel, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
    def m_func(self, X_train,X_test, y):
        n_samples, n_features = X_train.shape 
        nt_samples, nt_features= X_test.shape
        self.K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                self.K[i,j] = gaussian_kernel(X_train[i], X_train[j])
               # print(K[i,j])
        X_train=np.asarray(X_train)
        X_test=np.asarray(X_test)
        K1 = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K1[i,j] = gaussian_kernel(X_train[i], X_train[j])
               # print(K[i,j])
        print(K1.shape)
        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a_org=a
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        self.sv_yorg=y
        self.kernel = gaussian_kernel
        X_train=np.asarray(X_train)
        b = 0
        for n in range(len(self.a)):
            b += self.sv_y[n]
            b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        b /= len(self.a)
       # print(self.a_org[1])
        #print(self.a_org.shape,self.sv_yorg.shape,K.shape)
        w_phi=0
        total=0
        for n in range(len(self.a_org)):
            w_phi = self.a_org[n] * self.sv_yorg[n] * K1[n] 
        self.d_hyp=np.zeros(n_samples)
        for n in range(len(self.a_org)):
            self.d_hyp += self.sv_yorg[n]*(w_phi+b)
        func=np.zeros((n_samples))
        func=np.asarray(func)
        typ=2
        if(typ==1):
            for i in range(n_samples):
                func[i]=1-(self.d_hyp[i]/(np.amax(self.d_hyp[i])+0.000001))
        beta=0.2
        if(typ==2):
            for i in range(n_samples):
                func[i]=2/(1+beta*self.d_hyp[i])
        r_max=268/500
        r_min=1
        self.m=func[0:268]*r_min
        print(self.m.shape)
        self.m=np.append(self.m,func[268:768]*r_max)
        print(self.m.shape)
        
 ##############################################################################


    def fit(self, X_train,X_test, y):
        self.kernel = gaussian_kernel
        n_samples, n_features = X_train.shape 
        nt_samples, nt_features = X_test.shape
        # Gram matrix

        print(self.K.shape)

        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        for i in range(n_samples):
            sv=np.logical_or(self.a_org <self.m, self.a_org > 1e-5)
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        #print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        self.b /= len(self.a)
        print(self.b)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
        else :
            self.w = None        
        
    def project(self, X):
        if self.w is None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            X=np.asarray(X)
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * gaussian_kernel(X[i], sv)
                y_predict[i] = s
              #  print(y_predict[i])
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))

In [12]:

if __name__ == "__main__":
    import pylab as pl           
    def hyp_svm():
        
        clf = HYP_SVM(C=100.0)
        typ=2
        clf.m_func(X_train,X_test,y_train)
        clf.fit(X_train,X_test, y_train)
        y_predict = clf.predict(X_test)
        gm(y_predict,y_test)
        correct = np.sum(y_predict == y_test)
        print("%d out of %d predictions correct" % (correct, len(y_predict)))
        print("Accuracy",correct/len(y_predict))

    hyp_svm()    

(614, 614)
     pcost       dcost       gap    pres   dres
 0:  3.1997e+04 -2.6998e+06  3e+06  1e-01  3e-13
 1:  7.0493e+02 -3.7645e+05  4e+05  1e-02  2e-13
 2: -1.5080e+04 -8.7294e+04  7e+04  2e-03  2e-13
 3: -2.0698e+04 -4.2280e+04  2e+04  3e-04  3e-13
 4: -2.3448e+04 -3.5410e+04  1e+04  2e-04  3e-13
 5: -2.4919e+04 -3.1494e+04  7e+03  7e-05  3e-13
 6: -2.5867e+04 -2.9015e+04  3e+03  3e-05  3e-13
 7: -2.6304e+04 -2.8026e+04  2e+03  1e-05  3e-13
 8: -2.6625e+04 -2.7314e+04  7e+02  2e-06  3e-13
 9: -2.6775e+04 -2.7035e+04  3e+02  7e-07  3e-13
10: -2.6855e+04 -2.6900e+04  4e+01  4e-08  4e-13
11: -2.6872e+04 -2.6874e+04  2e+00  1e-09  4e-13
12: -2.6873e+04 -2.6873e+04  4e-02  2e-11  4e-13
13: -2.6873e+04 -2.6873e+04  6e-04  9e-13  4e-13
Optimal solution found.
optimal
(268,)
(614,)
(614, 614)
     pcost       dcost       gap    pres   dres
 0:  3.1997e+04 -2.6998e+06  3e+06  1e-01  3e-13
 1:  7.0493e+02 -3.7645e+05  4e+05  1e-02  2e-13
 2: -1.5080e+04 -8.7294e+04  7e+04  2e-03  2e-13
 3:

# Normal SVM using CVXOPT

In [ ]:
from cvxopt import matrix
class SVM(object):

    def __init__(self, kernel=gaussian_kernel, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
    def fit(self, X, y):
        self.kernel = gaussian_kernel
        n_samples, n_features = X.shape
        # Gram matrix
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = gaussian_kernel(X[i], X[j])
               # print(K[i,j])
        print(K.shape)

        P = cvxopt.matrix(np.outer(y,y) * K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
       # print(a)
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X[sv]
        self.sv_y = y[sv]
        print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * K[ind[n],sv])
        self.b /= len(self.a)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
                #print(self.w)
        else:
            self.w = None

    def project(self, X):
        if self.w is None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            X=np.asarray(X)
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * gaussian_kernel(X[i], sv)
                y_predict[i] = s
              #  print(y_predict[i])
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))

In [ ]:

if __name__ == "__main__":
    import pylab as pl           
    def normal_svm():
        
        clf = SVM(C=100.0)
        clf.fit(X_train, y_train)
        y_predict = clf.predict(X_test)
        gm(y_predict,y_test)
        correct = np.sum(y_predict == y_test)
        print("%d out of %d predictions correct" % (correct, len(y_predict)))
        print("Accuracy",correct/len(y_predict))

    normal_svm()    